# Importing  necessary modules

In [26]:
from time import sleep
from datetime import datetime
from kafka import KafkaConsumer
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
import sys
import random
import json
from json import loads

In [27]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegressionModel
from pyspark.sql.types import DoubleType

# Declearing spark context

In [28]:
sc.stop()
sc= SparkContext()
sqlContext = SQLContext(sc)

In [29]:

#vectorAssembler = VectorAssembler(inputCols=['open', 'high', 'low'], outputCol='features')
Path =  "StockPrice_Model"
load_model = LinearRegressionModel.load(Path)

In [31]:
try:
    consumer = KafkaConsumer('stock',bootstrap_servers=['localhost:9092'])
except:
    print('error')

# predicting values by using model 

In [32]:
def stock_prediction(load_model):
    for msg in consumer:
        res = json.loads(msg.value.decode('utf-8'))
        dlist = list(res.values())
        df = pd.DataFrame([dlist], columns=['Open', 'Close', 'Volume', 'High', 'Low'])
        df=df.astype(float)
        spark_df = sqlContext.createDataFrame(df)
        vectorAssembler = VectorAssembler(inputCols=['Open', 'High', 'Low'], outputCol='features')
        vectorAssembler = VectorAssembler(inputCols=['Open', 'High', 'Low'], outputCol='features')
        df_vect = vectorAssembler.transform(spark_df)
        df_vect_features = df_vect.select(['features','Close'])
        predictions = load_model.transform(df_vect_features)
        predictions.select("prediction", "Close", "features").show()
        predict_value = predictions.select('prediction').collect()[0].__getitem__("prediction")
        close_value = predictions.select('Close').collect()[0].__getitem__('Close')
        print(msg.key)
        date_time = msg.key.decode('utf-8')
        return predict_value, close_value, date_time

stock_prediction(load_model)

+------------------+------+--------------------+
|        prediction| Close|            features|
+------------------+------+--------------------+
|2534.0660651770945|1430.5|[1429.24,1430.425...|
+------------------+------+--------------------+

b'2020-06-05 10:25:00'


(2534.0660651770945, 1430.5, '2020-06-05 10:25:00')